#### What We will Learn

1. S3 Buckets --> Boto3 library
2. IAM Roles and Users
3. Complete Infrastructure of AWS Sagemaker ---> Bcoz here my entire training of the model will happen in AWS Sagemaker , along with that we will also try to create the Endpoint(and that too in the Sagemaker). U need not do the deployment in the EC2 instance and all(Bcoz AWS sagemaker provides that complete infrastructure).

In [ ]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

## First we will make boto3 client so that we will be able to communicate with the S3 bucket.
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name ## Region name : us-east-1 will be coming over here.
bucket = "mobbucketsagemaker" ## This is my S3 bucket which i really want to create.
print("Using bucket" + bucket)
## Similarily u can display ur region and everything.

In [ ]:
print(region) ## This will give u us-east-1

## Another Functionality
- SageMaker Runtime: Whenever u want to work with most of the services for deploying the model, training the model and all u basically use this boto3.client("sagemaker"). Sagemaker Runtime is basically used for inferencing.

In [12]:
df = pd.read_csv("mob_price_classification_train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1


- Here the entire training will not happen in my local system it will happen in the cloud itself.

In [13]:
df.shape

(2000, 21)

In [14]:
df.isnull().sum()  ## Here we are checking that there are null values or not.

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [15]:
## Since we are going to solve a classification problem we are going to check whether this dataset is imbalanced or not.
df['price_range'].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

- Since its not an imbalanced dataset we can directly use this. Otherwise i would have to make it balanced.

In [16]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [17]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [18]:
## Storing the last feature in the label --> which is my Price_Range
label = features.pop(-1)
label

'price_range'

In [ ]:
features ## Last feature price_range will be removed.

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi']

In [21]:
## Converting our features to X and y dataset.
X = df[features] ## Popping the last feature(price_range) as it belongs to my dependent_feature
y = df[label]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.15, random_state=0)

In [23]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


In [ ]:
## Converting X_train and X_test into a DataFrame
trainX = pd.DataFrame(X_train)
trainX[label] = y_train  ## So that i can save this files/data in my local or in my S3 bucket.

testX = pd.DataFrame(X_test)
testX[label] = y_test    ## So that i can save this files/data in my local or in my S3 bucket.

In [ ]:
trainX   ## This training data will also have my last feature which is price_range

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,4,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,14,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,13,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,4,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,20,48,1012,959,17,7,6,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1224,1,1.6,0,9,0,33,1.0,157,1,18,522,563,3796,10,5,13,1,1,0,3
1216,1158,0,0.7,1,1,1,29,0.7,123,2,20,311,1796,1542,17,9,15,1,0,1,1
1653,1190,0,2.0,1,0,0,40,0.2,93,5,1,1399,1646,3610,13,7,9,0,0,1,3
559,1191,0,2.4,1,2,0,13,0.9,169,1,7,179,1813,1028,14,6,8,1,1,1,0


- The reason why we are doing this is that i want to save this all in my S3 bucket and also want to save this in my csv file(locally).

In [26]:
## Saving in my CSV file
trainX.to_csv("train-V-1.csv",index=False)
testX.to_csv("test-V-1.csv",index=False)

In [ ]:
bucket ## See what is my bucket name : My bucket name is "mobbucketsagemaker"

- Whatever train and test data we have created we need to probably save this in my bucket.
- First we will see whether our bucket is created or not:
- In AWS search for S3 --> Click create bucket --> select bucket type as "General Purpose" enter the bucket name as "mobbucketsagemaker".(The bucket name should not be existing previously) Remember to block all the public access(go below it there)--> we need to just uncheck it. Then go below and click on "Create Bucket".
- Inside this bucket i will be uploading each and everything.

In [ ]:
## send the data to S3. and Sagemaker will take the data for training from S3
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer" ## Creating a prefix like inside what folder i need to go and upload that particular dataset
trainpath = sess.upload_data(path='train-V-1.csv',bucket=bucket, key_prefix=sk_prefix) ## Here we are giving the path of the local file where its present.

testpath = sess.upload_data(path='test-V-1.csv',bucket=bucket, key_prefix=sk_prefix)

print(trainpath)
print(testpath) ## Where ur entire csv files are getting uploaded. Note: upload_data() is a function that is available in boto3 it is basically going to upload the data to the particular location.

## We will be writing a script which will be used by AWS Sagemaker To Train Models
- Bcoz i m not going to run my model on the local machine.

In [ ]:
%%writefile script.py 
## Name of the script is script.py
## As soon as i execute this entire cell this will basically get converted to my script.py file.

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score
import sklearn
import joblib  ## To save our model in this format inside our s3 bucket
import boto3
import pathlib  ## So that we will be able to setup our path
from io import StringIO
import argparse  ## So that we provide the arguments in the runtime
import os
import numpy as np
import pandas as pd
## Once i execute this automatically my script.py file is created.

def model_fn(model_dir): ## This model directory is the same directory which will have my trained model path(Whatever path my model is saved after training that path i need to provide over here)
    clf = joblib.load(os.path.join(model_dir,"model.joblib")) ## First i have to load that particular model
    ## Here im joinging the path of model_dir and model.joblib so that i will be able to load it.

## Starting the execution of the script over here
if __name__ == "__main__":
    print("[Info] Extracting Arguments")
    ## Bcoz whenever we execute this in the AWS Sagemaker it definitely requires some kinds of Arguments
    ## Inorder to get the arguments i will write
    parser = argparse.ArgumentParser()

    ## Hyperparameter
    parser.add_argument("--n_estimators",type=int,default=100) ## This is my first argument
    parser.add_argument("--random_state",type=int,default=0)  ## So basically i m using some parameters and what is the default values
    ## If im passing in the argument that value will be set over here.
    ## This will be required by my AWS Sagemaker

    ## Data,model,and output directories
    ## This is required as sagemaker will be expecting these values like where my data is uploaded, where my model is, where my output directories is
    ## Why default sagemaker also creates some kind of environment variables
    ## Lets say once my training basically happens, so by default Sagemaker does one thing it creates a path and saves that particular model in that particular path.

    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR")) ## After the training is done by the sagemaker, automatically it will go and set up an environment variable which is called as "SM_MODEL_DIR" which will have the path where ur model is basically saved and it is also inside s3 only we will be seeing that particular path
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))  ## Next argument : After the model is basially trained in the AWS Sagemaker, this particular environment variable is also created over here
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))  ## This environment variable will also be created over there and it will have a value ---> This is also decided by AWS Sagemaker
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")  ## These are my default train and test file name

    ## This 3 pass are basically getting created in AWS-Sagemaker : "--model-dir", "--train", "--test"

    args,_ = parser.parse_known_args() ## This is the function where i will be able to get my known arguments itself.

    print("Sklearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    ## I will also be reading my train and test csv
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1) ## Removing the last column

    ## Building my train and test data
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order:')
    print(features)
    print()

    print("Label column is: ",label)
    print()

    print('Data Shape: ')
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()

    print("Training RandomForest Model ....") ## These all files will be running in my AWS Sagemaker
    print()
    model = RandomForestClassifier(n_estimator = args.n_estimator, random_state = args.random_state, verbose=2, n_jobs=1)
    model.fit(X_train,y_train)

    print()
    ## Now i will setup my model path so that i save that particular model path over here
    model_path = os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model, model_path)

    print("Model saved at" + model_path) ## This model_path is basically available in our S3 bucket

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ",X_test.shape[0])
    print("[TESTING] Model Accuracy is: ",test_acc)
    print("[TESTING] Testing Report: ")
    print(test_rep)



Writing script.py


### AWS Sagemaker Entry Point to Execute the Training script
- Because AWS needs to probably execute this and how we are going to provide script.py that we are going to see over here.

- Note : In the below code in the role part , search for IAM in the AWS account , in IAM go to roles(its a option just like user) --> Here we need to specify a role which has the access to the AWS Sagemaker execution(bcoz we are doing the execution from my coding environments).
- Click on "Create Role" --> Select trusted entity type: "AWS Service" , below in services select the service as sagemaker , then it will ask me whether we need to select : SageMaker Execution or SageMaker - HyperPod Clusters , we really need to select this first option(Sagemaker Execution) bcoz this allows to access S3, ECR, and cloudwatch(all these access u will be able to get it). Then click next. Again next.
- The next page --> Name, Review, and create , in role details it will ask me to provide Role Details , where we have to enter role details like Role Name , lets provide the name sagemakeraccess , then finally click create role.
- So here u will be able to see that sagemaker access role is created , from this u will be able to access everything like AWS S3 bucket, click that created role , in the summary u will have ARN url , this url will specifically be my role. We will copy this ARN and it will be assigned to my role in below code.
- In AWS Sagemaker also u will get different different instance access. Instance basically means what kind of machine.

In [ ]:
## Setup the entry point for the Sagemaker : Like how its going to pickup the script and all.
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION="0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py", ## This script.py will be responsible in executing my entire code
    role="",
    instance_count = 1,
    instance_type = "ml.m5.large", ## There are many instances like this(This instance will be used for training my entire script).
    framework_version = FRAMEWORK_VERSION,
    base_job_name = "RF-custom-sklearn",
    hyperparameters = {
        "n_estimators":100,
        "random_state":0
    },
    use_spot_instance = True, ## It is going to create the instance, its going to train the model and then its going to delete the instance
    max_run = 3600
)
## This is the entire configuration that we require in order to start my instance in the AWS Sagemaker


- Now i will launch my Training Job and this will happen Asynchronously(means we are just going to execute this in the backend and automatically this will get started)

In [ ]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train":trainpath, "test":testpath}, wait=True)
## As soon as i execute this my training will automatically get started in my AWS sagemaker
## First it will create an instance in my AWS Sagemaker, it will be creating a training-job with name: Will be shown , this training job will
## be available in my AWS Sagemaker

- Search for AWS Sagemaker in AWS account , there in sidebar u have something called as Training in training go to Training Job automatically u will see that training job has been created , when u click that training job u will be able to see the entire details.
- In the details , under Output Data Configuration u will see S3 output path this is where ur model will be saved.
- Search S3 bucket , inside S3 go and click on buckets there u will be able to see the folder that is basically created , click on the folder u will be able to see the model that is basically created. There when u click on the folder look for RF-custom-sklearn which u have given above. Click on it --> click source and there ur directory location will be there.

## Note:
- Whenever sagemaker does training its going to create the folder inside S3 buckets where ur model details will be there.

## Below code written is basically to get the model from S3

In [ ]:
sklearn_estimator.latest_training_job.wait(logs="None") ## From here i will get the latest training job
## Then i will probably tell my boto3 to describe this
artifact = sm_boto3.describe_training_job(
    TrainingJobName = sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]
## Whatever was my latest training job name i will take up the ModelArtifacts and S3ModelArtifacts(these are the keys that is basically present inside that).

- If i execute the above code i should be able to get my artifacts(artifact will probably provide u the entire information).

In [ ]:
## Printing my Artifact
artifact  

## Best thing about AWS Sagemaker
- Here on instance we are getting the instance, we are training the model, and closing it so that much more charges should not happen that is the best thing abt AWS Sagemaker.

## Next we will create the endpoint(in AWS Sagemaker) and from there we will do the inferencing.

## Deploy the Model for Endpoint
- Here we will discuss how we can probably deploy this model and create a endpoint.
- This should also happen in our AWS Sagemaker.

In [ ]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime,strftime  ## I want to create my model with recent time itself.

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name, ## Entire model will be created with this name into our endpoint
    model_data=artifact,
    role="Same what u have pasted above. That ARN URL",
    entry_point = "script.py",
    framework_version=FRAMEWORK_VERSION
)

In [ ]:
model   ## It will give output that : It is a type of sklearn model at this particular location

In [ ]:
## Endpoint Deployment
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count = 1, ## Bcoz for deployment also i require a seperate instance
    instance_type = "ml.m4.xlarge", ## This is the instance type that i m going to take for the deployment purpose and again it depends on the model size
    endpoint_name = endpoint_name
)
## Once we execute this model.deploy here our model will be deploying

- Now if u go to AWS Sagemaker and there u go to Inference , so there in inference u have something called as Endpoints , click it and here u will be able to see my endpoint is basically getting created. Along with endpoint we also have endpoint coniguration, u will be able to also see the information over here.
- I can use the above predictor variable and i can do any kind of prediction that i want. So like i can take records and directly do the prediction itself.
- Once the endpoint is generated click the endpoint and u will be able to see more info.

In [ ]:
testX[features]

In [ ]:
testX[features][0:2]  ## Here u will be able to see the first two records

In [ ]:
print(predictor.predict(testX[features][0:2].values.tolist())) ## Here i have to get two outputs saying that what category of price it basically follows
## Output : [3,0] --> here we did prediction for 1st two records i.e. for 1st record the output is 3 and for second record output is 0. i.e. 3rd category and 0th category

In [ ]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)
## Final thing is deleting the endpoint
## This is important bcoz : Otherwise ur endpoint will be created permanently i.e. it will keep on adding billing to ur AWS account
## This is one way of doing it from the code
## Other way : Search for Sagemaker , inside the sagemaker from the endpoint also i can close it up(Go to inferences --> There endpoints , select the endpoint , go to actions and delete the endpoint).
## Either u can do it from the code or u can do it from here.
## In sagemaker if u go to training --> training jobs this is basically opening the machine , training it and closing it from this u don't need to delete it.